# Imports

In [1]:
# data manipulation libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 60)

# data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize

%matplotlib inline
# to display visuals in the notebook

%config InlineBackend.figure_format='retina'
#to enable high resolution plots

# feature extraction and preprocessing
import re
import datetime

# feature transformation and preprocessing
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Auxiliary Functions

The function below will enable us to observe the missing values as a percentage per feature.

In [2]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent],
                              axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = (mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

# Understand & Clean & Format Data

In [3]:
train = pd.read_csv("../data/train/train.csv") 
test = pd.read_csv("../data/test/test.csv")
train.sample(5)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
6494,US,Very peppery even for Syrah. Marked also by in...,Bragger,87,20.0,California,Paso Robles,Central Coast,NaN,NaN,Wild Coyote 2009 Bragger Syrah (Paso Robles),Syrah,Wild Coyote
6754,France,The pure scent of freshly picked strawberries ...,Brut Rosé,89,22.0,Alsace,Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Wolfberger NV Brut Rosé Pinot Noir (Alsace),Pinot Noir,Wolfberger
3891,France,"Full, soft and rounded, this shows flavors of ...",NaN,89,35.0,Provence,Côtes de Provence,NaN,Roger Voss,@vossroger,Château d'Esclans 2011 Rosé (Côtes de Provence),Rosé,Château d'Esclans
3118,Italy,Roccolo Grassi takes Valpolicella Superiore to...,NaN,90,40.0,Veneto,Valpolicella Superiore,NaN,NaN,NaN,Roccolo Grassi 2009 Valpolicella Superiore,"Corvina, Rondinella, Molinara",Roccolo Grassi
272,US,"Though the alcohol comes close to 15%, this is...",Spofford Station,88,25.0,Oregon,Walla Walla Valley (OR),Oregon Other,Paul Gregutt,@paulgwine,Lujon 2007 Spofford Station Syrah (Walla Walla...,Syrah,Lujon


In [4]:
print("There are {} rows and {} columns in the train dataset."
      .format(train.shape[0], train.shape[1]))

There are 9000 rows and 13 columns in the train dataset.


In [5]:
print("There are {} rows and {} columns in the test dataset."
      .format(test.shape[0], test.shape[1]))

There are 1000 rows and 13 columns in the test dataset.


## Descriptive statistics & information about datasets

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                8994 non-null   object 
 1   description            9000 non-null   object 
 2   designation            6455 non-null   object 
 3   points                 9000 non-null   int64  
 4   price                  8403 non-null   float64
 5   province               8994 non-null   object 
 6   region_1               7505 non-null   object 
 7   region_2               3469 non-null   object 
 8   taster_name            7223 non-null   object 
 9   taster_twitter_handle  6888 non-null   object 
 10  title                  9000 non-null   object 
 11  variety                9000 non-null   object 
 12  winery                 9000 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 914.2+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                1000 non-null   object 
 1   description            1000 non-null   object 
 2   designation            716 non-null    object 
 3   points                 1000 non-null   int64  
 4   price                  920 non-null    float64
 5   province               1000 non-null   object 
 6   region_1               831 non-null    object 
 7   region_2               384 non-null    object 
 8   taster_name            792 non-null    object 
 9   taster_twitter_handle  756 non-null    object 
 10  title                  1000 non-null   object 
 11  variety                1000 non-null   object 
 12  winery                 1000 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 101.7+ KB


In [8]:
train.describe()

,points,price
count,9000.000000,8403.000000
mean,88.455222,35.532191
std,3.025945,40.750683
min,80.000000,5.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,42.000000
max,100.000000,1300.000000


Majority of the features are categorical and we have some missing data in the both datasets. Machine learning models can only work with numerical and non-empty values. Thus we are going to develop strategies in Feature Engineering to impute the missing data and transform categorical values into the numeric values.

In [9]:
test.describe()

,points,price
count,1000.000000,920.000000
mean,88.503000,34.675000
std,3.067475,42.240874
min,80.000000,7.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,41.000000
max,97.000000,1000.000000


## Description of features and target

In [10]:
train.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,US,"Expressive aromas of smoke, embers and blue fr...",NaN,88,35.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Damsel 2013 Syrah (Columbia Valley (WA)),Syrah,Damsel
1,South Africa,"Soft mint, spice, cocoa and smoke on the nose ...",Redhill,89,30.0,Stellenbosch,NaN,NaN,Susan Kostrzewa,@suskostrzewa,Simonsig 2005 Redhill Pinotage (Stellenbosch),Pinotage,Simonsig
2,Portugal,"An elegant, finely rounded wine, with firm tan...",NaN,90,NaN,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta de la Rosa 2008 Red (Douro),Portuguese Red,Quinta de la Rosa
3,South Africa,Winemaker: Louis Nel. This Cab-Shiraz blend is...,Cape Winemakers Guild Rapscallion,91,NaN,Stellenbosch,NaN,NaN,Lauren Buzzeo,@laurbuzz,Louis Nel 2015 Cape Winemakers Guild Rapscalli...,Cabernet Sauvignon-Shiraz,Louis Nel
4,Portugal,"Lightly wood aged and spicy, this is a fine re...",Casa Américo Branco Reserva,90,NaN,Dão,NaN,NaN,Roger Voss,@vossroger,Seacampo 2014 Casa Américo Branco Reserva Encr...,Encruzado,Seacampo


In [11]:
train.sample(5)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1686,Argentina,This ripe Malbec offers jammy blueberry aromas...,La Madras Vineyard,90,20.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Ricardo Santos 2013 La Madras Vineyard Malbec ...,Malbec,Ricardo Santos
5999,France,An astonishing nose of peach and bramble with ...,Herrenweg De Turckheim,92,50.0,Alsace,Alsace,NaN,NaN,NaN,Domaine Zind-Humbrecht 2007 Herrenweg De Turck...,Pinot Gris,Domaine Zind-Humbrecht
3272,US,"Youthfully wound up, this wine celebrates the ...",40th Anniversary Estate Grown,91,50.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Grgich Hills 2014 40th Anniversary Estate Grow...,Chardonnay,Grgich Hills
6774,France,Menetou-Salon is a near neighbor of Sancerre. ...,La Charnivolle,86,19.0,Loire Valley,Menetou-Salon,NaN,Roger Voss,@vossroger,Fournier Père et Fils 2006 La Charnivolle (Me...,Sauvignon Blanc,Fournier Père et Fils
2397,US,Tangy cranberry fruit carries light impression...,Cattrall Brothers Vineyard,90,48.0,Oregon,Eola-Amity Hills,Willamette Valley,Paul Gregutt,@paulgwine,Big Table Farm 2014 Cattrall Brothers Vineyard...,Pinot Noir,Big Table Farm


With some intuition, expert knowledge and help of Google here are the explanation of features and target:

- <b>country:</b> Origin of the wine producer
- <b>description:</b> Presentment of the taster to describe the wine
- <b>designation:</b> Name of the wine given to the wine by the producer, sometimes used interchangeably with vineyard. Usually available in the title.
- <b>points:</b> Our target value, representing score of a particular wine received from a taster. An important remark is, a wine may receive different points from the same taster.
- <b>price:</b> Price of the wine
- <b>region_1:</b> Official definition of the place where the grapes for a wine are grown
- <b>region_2:</b> Official definition of the place where the grapes for a wine are grown
- <b>taster_name:</b> The taster, as obvious assigns points to the wine
- <b>title:</b> Name of the wine, as available on the label
- <b>variety:</b> Grape variety of the wine
- <b>winery:</b> name of the wine producer

## Missing values of the datasets

In [12]:
missing_values_table(train)

Your selected dataframe has 13 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
region_2,5531,61.5
designation,2545,28.3
taster_twitter_handle,2112,23.5
taster_name,1777,19.7
region_1,1495,16.6
price,597,6.6
country,6,0.1
province,6,0.1


In [13]:
missing_values_table(test)

Your selected dataframe has 13 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
region_2,616,61.6
designation,284,28.4
taster_twitter_handle,244,24.4
taster_name,208,20.8
region_1,169,16.9
price,80,8.0


Features that have missing values more than 50% will be dropped since they aren't likely to provide significant information to the machine learning model.

In this sample dataset, target value has no missing values, if we had some, we would drop all the missing values of the target to not to leak any data or distort the existing distribution of the target.

Considering above comments and looking at the missing values, region_2 will be dropped.

## Cardinality of the non-numeric features

High cardinality features may cause problems when we are training our model. Even though we transform each unique value of a categorical feature to a new feature, they may bring the curse of dimensionality. So, we are going to use the high cardinality features for adding new features, then we are going to remove them from the datasets.

In [14]:
object_columns = (train
                  .select_dtypes(include="object")
                  .columns)

for column in object_columns:
    print("{} has {} unique values."
          .format(column, 
                  train[column]
                  .nunique()))

country has 37 unique values.
description has 8948 unique values.
designation has 4913 unique values.
province has 236 unique values.
region_1 has 699 unique values.
region_2 has 17 unique values.
taster_name has 18 unique values.
taster_twitter_handle has 14 unique values.
title has 8937 unique values.
variety has 325 unique values.
winery has 5097 unique values.


<b>Cardinality</b> is the number of unique values that a feature has.

<b>We have some high-cardinality features:</b>

- description
- designation
- title
- winery

<b>We have some moderate-cardinality features:</b>

- province
- region_1
- variety

<b>We have some low-cardinality features:</b>

- country
- taster_name
- taster_twitter_handle

Description contains information about wine's color, taste and notes (like citrus, tannins). Title feature contains year of the wine produced. Variety has the information about if different type of grapes are blended or not. They can be used to extract more features.

High cardinality features will be dropped from the dataset, if not used for feature extraction.

Most frequent values of moderate and low cardinality features will be explored in the Exploratory Data Analysis.

## Unique values of taster_name and taster_twitter_handle

In [15]:
taster_names = train["taster_name"].unique()

for value in taster_names:
    print(value)

Sean P. Sullivan
Susan Kostrzewa
Roger Voss
Lauren Buzzeo
Joe Czerwinski
Michael Schachner
Jim Gordon
Kerin O’Keefe
Matt Kettmann
nan
Paul Gregutt
Virginie Boone
Anna Lee C. Iijima
Jeff Jenssen
Mike DeSimone
Anne Krebiehl MW
Alexander Peartree
Fiona Adams
Carrie Dykes


In [16]:
taster_twitter_handles = (train["taster_twitter_handle"]
                          .unique())

for value in taster_twitter_handles:
    print(value)

@wawinereport
@suskostrzewa
@vossroger
@laurbuzz
@JoeCz
@wineschach
@gordone_cellars
@kerinokeefe
@mattkettmann
nan
@paulgwine 
@vboone
@worldwineguys
@AnneInVino
@bkfiona


The objective is to show good prediction is possible using wine predictor, which is a machine learning model itself. A "good prediction" is possible with features of independent variables that are predictive of the points. 

So we do not want to use dependent features (like taster_twitter_handle and the taster_name) when building our model, because they are providing the same information to the model and possibly making it unnecessarily complicated. 

After some online research and making sure that each unique value of taster_twitter_handle is available in the taster_name feature, we can remove taster_twitter_handle to build a good predictor.